In [1]:
import torch
import gpt1

torch.manual_seed(10)

# GPT



In [2]:
device = 'cpu'
batch_size = 2
seq_len = 40

max_seq_len = 300
vocab_size = 100
emb_size = 128

head_size = 4
num_heads = 3
num_layers = 5
dropout = 0.3

In [3]:
# на входе тензор токенов (batch_size,seq_len)
x = torch.randint(low=0,high=vocab_size-1,size=(batch_size,seq_len), dtype=torch.long, device=device)
x.shape, x.dtype

(torch.Size([2, 40]), torch.int64)

In [4]:
gpt = gpt1.GPT(
    vocab_size=vocab_size,
    max_seq_len=max_seq_len,
    emb_size=emb_size,
    num_heads=num_heads,
    head_size=head_size,
    num_layers=num_layers,
    dropout=dropout,
    device=device)

In [5]:
#  на выходе loggits batch_size x seq_len x vocab_size
y = gpt.forward(x)
y.shape, y.dtype, torch.isnan(y).any().item()

(torch.Size([2, 40, 100]), torch.float32, False)

# GPT.generate

In [17]:
max_new_tokens = 20
#  на выходе []int batch_size x (seq_len + max_new_tokens)
new_seq = gpt.generate(x=x,max_new_tokens=max_new_tokens)
new_seq.shape, new_seq.dtype, torch.isnan(new_seq).any().item(), torch.unique(new_seq)

(torch.Size([2, 60]),
 torch.int64,
 False,
 tensor([ 1,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 15, 18, 20, 21, 22, 23, 24,
         25, 26, 27, 32, 33, 34, 35, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51,
         52, 53, 54, 55, 57, 58, 60, 62, 63, 65, 67, 68, 69, 70, 74, 79, 80, 81,
         82, 83, 84, 85, 89, 90, 94, 95]))

In [7]:
# заготовка, чтоб посмотреть как работает softmax
t = torch.rand([2,4,3])
t.shape,t.dtype

(torch.Size([2, 4, 3]), torch.float32)

In [8]:
# сумма по каждой строке 1
prob = torch.softmax(t,dim=-1)
prob[0]

tensor([[0.4062, 0.4031, 0.1907],
        [0.3458, 0.4506, 0.2036],
        [0.2925, 0.3295, 0.3780],
        [0.5210, 0.2427, 0.2363]])

In [9]:
0.4062+ 0.4031+ 0.1907

1.0

In [10]:
seq = torch.argmax(prob,dim=-1,keepdim=True)
seq, seq.shape

(tensor([[[0],
          [1],
          [2],
          [0]],
 
         [[1],
          [2],
          [2],
          [1]]]),
 torch.Size([2, 4, 1]))